In [1]:
from jupyter_plotly_dash import JupyterDash

import base64

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "CodeJunkie"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Graz_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(
            style={'display': 'flex'},
            children=[
                html.Div(html.H3('Jacob McPherson, DataTable and GeoLocation',
                                 style={'textAlign': 'right', 'color': '#6900D8'})),
                html.Div(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                                  style={'height': '1' '30px'}))
            ]),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='radio-items-id',
            options=[{'label': 'Water', 'value': 'W'},
            {'label': 'Mountain/Wilderness', 'value': 'MW'},
            {'label': 'Disaster/IndividualTracking', 'value': 'DIT'},
            {'label': 'Reset', 'value': 'R'}],
            labelStyle={'display': 'inline-block'}
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'minWidth': '130px', 'width': '180px', 'maxWidth': '180px',
            'whiteSpace': 'normal'
        },
        #user friendly feature setup
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('radio-items-id', 'value')])
def update_dashboard(value):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if value == 'W':
        df = pd.DataFrame.from_records(shelter.read({"$and": [{"$or":
            [{"breed": "Labrador Retriever Mix"}, {"breed": "Chesapeake Bay Retriever"}, {"breed": "Newfoundland"}]},
            {"sex_upon_outcome": "Intact Female"}, {"age_upon_outcome_in_weeks": {"$gte":26, "$lte":156}}]}))
    
    if value == 'MW':
        df = pd.DataFrame.from_records(shelter.read({"$and": [{"$or":
        [{"breed": "German Shepherd"}, {"breed": "Alaskan Malamute"}, {"breed": "Old English Sheepdog"},
         {"breed": "Siberian Husky"}, {"breed": "Rottweiler"}]},
        {"sex_upon_outcome": "Intact Male"}, {"age_upon_outcome_in_weeks": {"$gte":26, "$lte":156}}]}))
        
    if value == 'DIT':
        df = pd.DataFrame.from_records(shelter.read({"$and": [{"$or":
        [{"breed": "Doberman Pinsch"}, {"breed": "German Shepherd"}, {"breed": "Golden Retriever"},
         {"breed": "Bloodhound"}, {"breed": "Rottweiler"}]},
        {"sex_upon_outcome": "Intact Male"}, {"age_upon_outcome_in_weeks": {"$gte":20, "$lte":300}}]}))
        
    if value == 'R':
        df = pd.DataFrame.from_records(shelter.read({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
            
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure ={
                "data": [{"x": dff["name"], "y": dff["breed"], "type": "bar",
                          "marker": {"color": ['red','blue','green','orange','purple','yellow','black','brown',
                                               'pink', 'silver']}}],
                "layout": {
                    "xaxis": {"automargin": True},
                    "yaxis": {"automargin": True, "title": {"text": "First bar graph"}},
                    "height": 250,
                    "margin": {"t": 10, "1": 10, "r": 10}
                }
            }
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[dff.loc[0, 'location_lat'], dff.loc[0, 'location_long']], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0, 'location_lat'], dff.loc[0, 'location_long']], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
                ])
            ])
        ])
    ]


app